In [25]:
import pandas as pd
import numpy as np
import requests
import time

### API

In [170]:
def get_submissions(subreddit, n_iter, epoch_right_now):
    base_url = 'https://api.pushshift.io/reddit/search/submission?subreddit='
    
    df_list = []
    
    current_time = epoch_right_now
    
    for post in range(n_iter):
        res = requests.get(base_url,params = {'subreddit':subreddit,'size':100, 'lang':100, 'before':current_time,'selftext:not':'[removed]','q:not':'[deleted]'})
    
        df = pd.DataFrame(res.json()['data'])
        
        df= df.loc[:, ['title','created_utc','selftext','subreddit','author','media_only','permalink']]
        
        df_list.append(df)
        
        # sleep 5 seconds after each request
        time.sleep(5)
        
        current_time = df['created_utc'].min()
        
    return pd.concat(df_list, axis=0)

---
### get the data

In [215]:
math_submissions = get_submissions('math',50,1617047153)

In [264]:
math_submissions.to_csv('math_submissions.csv',index=False)

In [171]:
physics_submissions = get_submissions('Physics',50,1617047153)

In [172]:
physics_submissions.to_csv('physics_submissions.csv',index=False)

### Take a look of the removed post, comparing with the data wihout removed posts

In [609]:
# get removed posts, the previous API exclude removed data, 
# this API will only include removed API
def get_submissions_removed(subreddit, n_iter, epoch_right_now):
    base_url = 'https://api.pushshift.io/reddit/search/submission?subreddit='
    
    df_list = []
    
    current_time = epoch_right_now
    
    for post in range(n_iter):
        res = requests.get(base_url,params = {'subreddit':subreddit,'size':100, 'lang':100, 'before':current_time,'selftext':'[removed]'})
    
        df = pd.DataFrame(res.json()['data'])
        
        df= df.loc[:, ['title','created_utc','selftext','subreddit','author','media_only','permalink']]
        
        df_list.append(df)
        
        time.sleep(5)
        
        current_time = df['created_utc'].min()
        
    return pd.concat(df_list, axis=0)

math_submissions_removed =  get_submissions_removed('math',50,1617047153)

In [610]:
math_submissions_removed.to_csv('math_submissions_removed.csv',index=False)

---